In [ ]:
from ipywidgets import Layout, Button, Box, Label, Text, Output, Tab
from IPython.display import display
from timeStamp import rfidTimeStamp as rfidModule
import sqlite3 
import time
import datetime

class timeRegistration(rfidModule):
    '''
        timeRegistration is a class that contains the UI. Functionality:
        
        1) Create new users in the data base
        2) Modify time registration of existing users. This function is made to allow write into the database should the user
        accidentially forget to log in or log out
        3) Users time registration data can be extracted within time period and allow report generation followed by sending report over email. 
    '''
    def __init__(self, dbPath):
        
        #Connect to the database
        self.dbPath = dbPath
        self.createDatabaseIfNotExist(dbPath)
        
        #Timeout meaning how long time to wait for an action before going into idle mode
        self.timeout = 10 #in sec
        self.timeFormat = "%Y%m%d%H%M%S"
        
        #Initializing the GUI
        self.buildGUI()
    
    def buildGUI(self):
        
        #REGISTER NEW EMPLOYEE
        form_item_layout = Layout(
        display = 'flex',
        flex_flow = 'row',
        justify_content = 'space-between')
        
        self.employeeRegNameText = Text(placeholder = 'Employee full name', disabled = False)
        self.employeeRegButton = Button(description = 'Register', disabled = False, 
                                        button_style = 'success',
                                        tooltip = 'Register new employee card')
        
        self.employeeRegButton.on_click(self.registerNewEmployee)
        
        self.employeeRegOut = Output()
        
        self.employeeRegFormItems = [
            Box([Label(value = 'New employee name* : '), self.employeeRegNameText]),
            Box([self.employeeRegButton]),
            Box([self.employeeRegOut])
        ]
        
        self.registerForm = Box(self.employeeRegFormItems, layout = Layout(
        display = 'flex',
        flex_flow = 'column',
        border = 'solid 2px',
        align_items = 'stretch',
        width = '70%'))
        
        #TAB
        self.tab = Tab()
        self.tab.children = [self.registerForm]
        self.tab.set_title(0, 'Register')
        
        display(self.tab)
        
    def createDatabaseIfNotExist(self, databasePath):
        '''
        Check if the table of the data base provided by the databasePath exists. If not it will be created.
        '''
        conn = sqlite3.connect(databasePath)
        sqlString = '''CREATE TABLE IF NOT EXISTS workinghour (name text, employeeNo text, datetime text, status text)'''
        c = conn.cursor()
        c.execute(sqlString)
        conn.commit()
        
        sqlString = '''CREATE TABLE IF NOT EXISTS employee (name text, employeeNo)'''
        c.execute(sqlString)
        conn.commit()
        conn.close()
    
    def connectDataBase(self):
        '''
        connects to the sql data base
        '''
        conn = sqlite3.connect(self.dbPath)
        c = conn.cursor()
        return conn, c
        
    def writeEmployeeToDb(self, cardString):
        '''
            Register the employee into sql data base. The cardString is a string from the card in the format
            employee name;employee number; check in status
        '''
        cardDict = self.splitCardString(cardString)
        sqlString = f''' INSERT INTO employee(name, employeeNo)
              VALUES("{cardDict['name']}", "{cardDict['employeeNumber']}")'''
        
        conn,c = self.connectDataBase()
        c.execute(sqlString)
        conn.commit()
        conn.close()
            
    def registerNewEmployee(self, change):
        #Extract employee full name from the text
        employeeName = self.employeeRegNameText.value
        if not employeeName:
            with self.employeeRegOut:
                self.employeeRegOut.clear_output()
                print('Error! Employee name is blank!')
                return
            
        #Generate a unique employee number. The number is based on date of employment.
        now = datetime.datetime.now()
        employeeNumber = now.strftime(self.timeFormat)
        checkInStatus = 'registered' #First time when the employee is registered, the check in status is set to registered
        #put employee name, employee number and checkin status into one string and write it to the card
        stringToCard = employeeName.strip() + ';' + employeeNumber.strip() + ';' + checkInStatus
        
        with self.employeeRegOut:
            self.employeeRegOut.clear_output()
            print('Ready to register ' + employeeName + ' swipe the card to the reader')
            idnum, textin = self.writeCard(stringToCard, timeout = self.timeout)
            
            if idnum is not None:
                #Write the employee into the data base
                self.writeEmployeeToDb(stringToCard)
                print(employeeName + ' has been registered.')
                print('Employee number is ' + employeeNumber)
                print('Complete!')
            else:
                print('Timeout ' + employeeName + ' not registered. Try again...')
            

obj = timeRegistration('employeeHour.db')

In [3]:
import sqlite3
conn = sqlite3.connect('employeeHour.db')
c = conn.cursor()
c = conn.execute('select * from employee')
data = c.fetchall()
print(data)
conn.close()

[('Oliver Wu', '20210107190143'), ('Elin Wu', '20210107190258')]


In [ ]:

GPIO.cleanup()